In [1]:
import Levenshtein
from itertools import islice

In [2]:
def readFastq(fastqfile):
    "parse a fastq-formatted file, yielding a (header, sequence, quality) tuple"
    fastqiter = (l.strip('\n') for l in fastqfile)  # strip trailing newlines 
    fastqiter = filter(lambda l: l, fastqiter)  # skip blank lines
    while True:
        fqlines = list(islice(fastqiter, 4))
        if len(fqlines) == 4:
            header1,seq,header2,qual = fqlines
        elif len(fqlines) == 0:
            return
        else:
            raise EOFError("Failed to parse four lines from fastq file!")

        if header1.startswith('@') and header2.startswith('+'):
            yield header1[1:], seq, qual
        else:
            raise ValueError("Invalid header lines: %s and %s for seq %s" % (header1, header2, seq))

COMP_TABLE = str.maketrans("ACTGN", "TGACN")

def revcomp(seq):
    '''Return the reverse complement of a DNA sequence'''
    if not set(seq).issubset({'A', 'C', 'G', 'T', 'N'}):
        raise ValueError(f"Sequence ({seq}) must only contain ACTGN")
    return seq.translate(COMP_TABLE)[::-1]

def all_kmers(seq, k):
    return [seq[i:i + k] for i in range(len(seq) - k + 1)]

def first_index_of_kmer(seq, kmer, dist, dist_func):
  k = len(kmer)
  kmers = all_kmers(seq, k)
  for i, s in enumerate(kmers):
    if dist_func(kmer, s) <= dist:
      return i
  return -1

def best_index_of_kmer(seq, kmer, dist, dist_func):
  k = len(kmer)
  kmers = all_kmers(seq, k)
  lowest_distance = dist + 1
  best_index = -1
  for i, s in enumerate(kmers):
    distance = dist_func(kmer, s)
    if distance <= dist and distance < lowest_distance:
      lowest_distance = distance
      best_index = i
  return best_index

In [3]:
QUERY = 'ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGGCATTTTGCCTTCCTGTTTTTGCTCACCCAGAAACGCTGGTGAAAGTAAAAGATG'

In [4]:
# read in sequences from fastq
with open('Lab_a3b_1_MORF.fastq', 'r') as f:
    parsed_fastq = readFastq(f)
    reads = [('>' + header, seq) for header, seq, _ in parsed_fastq]

# concatenate each sequence to itself in case the read starts in the middle of the insert site
reads_concat = [(header, seq + seq) for header, seq in reads]

# get first instance of upstream sequence and the sequence downstream of it
reindexed_seqs = []
for header, seq in reads_concat:
    index = first_index_of_kmer(seq, QUERY, 10, Levenshtein.distance)
    if index >= 0:
        reindexed_seq = seq[index:index + int(len(seq)/2)]
        reindexed_seqs.append((header, reindexed_seq))
    else:
        index_rev = first_index_of_kmer(revcomp(seq), QUERY, 10, Levenshtein.distance)
        if index_rev >= 0:
            reindexed_seq = revcomp(seq)[index_rev:index_rev + int(len(seq)/2)]
            reindexed_seqs.append((header, reindexed_seq))

In [5]:
print(len(reindexed_seqs))

166


In [6]:
with open('MORF_reindexed.fasta', 'w') as f:
    for header, seq in reindexed_seqs:
        f.write(f'{header}\n{seq}\n')